In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats import proportion
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 1. Загрузка и первичный анализ данных

In [2]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
# Загрузка данных с правильным именем файла
data = pd.read_excel(r'C:\Users\leras\Downloads\Датасет.xlsx', header=1)
print(f"Данные загружены. Размер: {data.shape}")
print(f"Колонки: {list(data.columns)}")
data.head()

Данные загружены. Размер: (12281, 7)
Колонки: ['id_order', 'test_group', 'time_order', 'time_offer', 'time_assign', 'time_arrive', 'trip_time']


,id_order,test_group,time_order,time_offer,time_assign,time_arrive,trip_time
0,10001,0,04.3.2024 0:00:01,04.3.2024 0:00:42,04.3.2024 0:01:13,04.3.2024 0:09:07,04.3.2024 1:27:28
1,10002,0,04.3.2024 0:00:22,04.3.2024 0:01:00,04.3.2024 0:01:14,04.3.2024 0:08:45,04.3.2024 0:22:22
2,10003,1,04.3.2024 0:07:08,04.3.2024 0:07:44,NaN,NaN,NaN
3,10004,1,04.3.2024 0:07:11,04.3.2024 0:08:21,04.3.2024 0:08:46,04.3.2024 0:21:13,04.3.2024 1:23:25
4,10005,0,04.3.2024 0:11:59,04.3.2024 0:12:50,04.3.2024 0:13:09,04.3.2024 0:21:43,04.3.2024 0:46:45


# 2. Предобработка и очистка данных

In [6]:
# Проверка структуры данных
print("Информация о данных:")
print(data.info())
print("\nПропущенные значения:")
print(data.isnull().sum())
print("\nРаспределение по группам:")
print(data['test_group'].value_counts())

Информация о данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12281 entries, 0 to 12280
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id_order     12281 non-null  int64 
 1   test_group   12281 non-null  int64 
 2   time_order   12281 non-null  object
 3   time_offer   10781 non-null  object
 4   time_assign  9624 non-null   object
 5   time_arrive  7724 non-null   object
 6   trip_time    7384 non-null   object
dtypes: int64(2), object(5)
memory usage: 671.7+ KB
None

Пропущенные значения:
id_order          0
test_group        0
time_order        0
time_offer     1500
time_assign    2657
time_arrive    4557
trip_time      4897
dtype: int64

Распределение по группам:
test_group
1    6597
0    5684
Name: count, dtype: int64


In [7]:
# Преобразование дат
date_columns = ['time_order', 'time_offer', 'time_assign', 'time_arrive', 'trip_time']
for col in date_columns:
    data[col] = pd.to_datetime(data[col], format='%d.%m.%Y %H:%M:%S', errors='coerce')

print("Даты преобразованы")
print(data.info())

Даты преобразованы
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12281 entries, 0 to 12280
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id_order     12281 non-null  int64         
 1   test_group   12281 non-null  int64         
 2   time_order   12281 non-null  datetime64[ns]
 3   time_offer   10781 non-null  datetime64[ns]
 4   time_assign  9624 non-null   datetime64[ns]
 5   time_arrive  7724 non-null   datetime64[ns]
 6   trip_time    7384 non-null   datetime64[ns]
dtypes: datetime64[ns](5), int64(2)
memory usage: 671.7 KB
None


# 3. Проверка корректности A/B сплитования

In [8]:
# Проверка корректности A/B сплитования
print("Проверка сплит-системы:")
print(f"Всего заказов: {len(data)}")
print(f"Группа 0 (контрольная): {len(data[data['test_group']==0])} ({len(data[data['test_group']==0])/len(data)*100:.1f}%)")
print(f"Группа 1 (тестовая): {len(data[data['test_group']==1])} ({len(data[data['test_group']==1])/len(data)*100:.1f}%)")

# Проверка случайности распределения по времени
data['order_hour'] = data['time_order'].dt.hour
hour_distribution = data.groupby(['order_hour', 'test_group']).size().unstack()
print("\nРаспределение по часам:")
print(hour_distribution.head())

Проверка сплит-системы:
Всего заказов: 12281
Группа 0 (контрольная): 5684 (46.3%)
Группа 1 (тестовая): 6597 (53.7%)

Распределение по часам:
test_group    0    1
order_hour          
0            25   29
1           124  135
2            90  101
3           107  145
4           116  144


# 4. Анализ воронки заказов

In [9]:
# Анализ воронки заказов
def calculate_funnel(data):
    funnel = {
        'orders': len(data),
        'offers': data['time_offer'].notna().sum(),
        'assigns': data['time_assign'].notna().sum(),
        'arrives': data['time_arrive'].notna().sum(),
        'trips': data['trip_time'].notna().sum()
    }
    return funnel

funnel_total = calculate_funnel(data)
funnel_0 = calculate_funnel(data[data['test_group']==0])
funnel_1 = calculate_funnel(data[data['test_group']==1])

print("Воронка заказов - все данные:")
for step, count in funnel_total.items():
    print(f"{step}: {count}")

print("\nВоронка по группам:")
print("Группа 0 (3km):", funnel_0)
print("Группа 1 (5km):", funnel_1)

Воронка заказов - все данные:
orders: 12281
offers: 10781
assigns: 9624
arrives: 7724
trips: 7384

Воронка по группам:
Группа 0 (3km): {'orders': 5684, 'offers': np.int64(4846), 'assigns': np.int64(4285), 'arrives': np.int64(3689), 'trips': np.int64(3562)}
Группа 1 (5km): {'orders': 6597, 'offers': np.int64(5935), 'assigns': np.int64(5339), 'arrives': np.int64(4035), 'trips': np.int64(3822)}


# 5. Статистические тесты и значимость

# 6. Выявление отложенных заказов

# 7. Финальные выводы и рекомендации